# Import libraries

In [8]:
import requests as req
import pandas as pd
import os
from functools import reduce
import json
import logging
import sys

logging.basicConfig(stream=sys.stderr, level=logging.INFO)

# Authenticate to API

In [31]:
from configparser import ConfigParser
config = ConfigParser()
config.read("/Users/nathansuberi/Desktop/WRI_Programming/cred/.env")
api_token = config.get("auth", "rw_api_token")

AUTH_TOKEN = api_token # <Insert Auth Token Here>

# Download upload config

In [59]:
#### Download Google Spreadsheets ####
# ImageCollection Upload Config
!curl "https://docs.google.com/spreadsheets/d/1NCL998eESaXcodvzTE7e9_zpBxVAG4HiQXpzP1E_Cpk/export?format=tsv" > ic_upload_config.tsv
ic_upload_config = pd.read_csv(open("ic_upload_config.tsv", "r"), sep="\t", index_col=None)
os.remove("ic_upload_config.tsv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   667    0   667    0     0    683      0 --:--:-- --:--:-- --:--:--   684


In [60]:
ic_upload_config

,wri_id,imageCollectionName,datasetName,SLD
0,foo_051,users/resourcewatch_wri/foo_051_vegetation_con...,Vegetation Condition Index (NDVI anomalies),"<RasterSymbolizer> <ColorMap type=""intervals"" ..."


# Helper functions

In [20]:
def createHeaders():
    return {
        'content-type': "application/json",
        'authorization': "Bearer {}".format( AUTH_TOKEN )
    }

def upload_ic_to_backoffice(wri_id, imageCollectionName, datasetName):
    
    ds_specs = {
        "connectorType":"rest",
        "provider":"gee",
        "tableName":imageCollectionName,
        "application":["rw"],
        "geoInfo":True,
        "type":"raster",
        "name":"{}_{}".format(wri_id, datasetName)
    }

    create_res = req.request("POST", 
                      'https://staging-api.globalforestwatch.org/v1/dataset', 
                      data=json.dumps(ds_specs), 
                      headers = createHeaders())

    logging.info(create_res.text)

    return create_res.json()['data']['id']


def autogen_layer_def(imageCollectionName, layerName, sld):  
    
    layer_template = {
        "name":layerName,
        "application":["rw"],"iso":[],
        "default":True,
        "protected":False,
        "layerConfig":{
            "type":"gee",
            "position":"last",
            "isImageCollection":True,
            #"filterDates":["2010-01-01","2017-10-24"],
            "assetId":imageCollectionName,
            "body":{
                "styleType":"sld",
                "sldValue":sld
            }},
        "legendConfig":{},
        "interactionConfig":{},
        "applicationConfig":{},
        "staticImageConfig":{},
    }
    
    #layer_template['layerConfig']['timeline'] = True
    #layer_template['layerConfig']['order'] = int(str_year)
    #layer_template['layerConfig']['timelineLabel'] = str(str_year)
   
    return layer_template
        
def upload_layer_def_to_backoffice(layer_def, rw_id):
    url = "https://resourcewatch.org/v1/dataset/{}/layer".format(rw_id)
    res = req.request("POST", url, data=json.dumps(layer_def), headers = createHeaders())
    return res.text

def prepare_data(obj, info):
    wri_id = info[0]
    imageCollectionName = info[1]
    datasetName = info[2]
    sld = info[3]
        
    rw_id = upload_ic_to_backoffice(wri_id, imageCollectionName, datasetName)
    layer_def = autogen_layer_def(imageCollectionName, 'Most recent ' + datasetName, sld)
    layer_on_backoffice = upload_layer_def_to_backoffice(layer_def, rw_id)
    
    #layer_defs = list(map(lambda year: autogen_layer_def(year, max_year, rw_id, carto_name, cartocss, legend, interaction, basesql), wbg_ds_georef['year'].unique()))
    #logging.info(layer_defs)
    #layer_defs_on_backoffice = list(map(lambda ldef: upload_layer_defs_to_backoffice(ldef, rw_id), layer_defs))
    #logging.info(layer_defs_on_backoffice)
    #vocab_on_backoffice = load_tags_to_backoffice(tags, rw_id)
    #logging.info(vocab_on_backoffice)
    
    obj[rw_id] = {
        'wri_id':wri_id,
        'layers':json.loads(layer_on_backoffice),
        #'tags':vocab_on_backoffice
    }
    
    return obj

In [33]:
def slr_layer(ix, amt):
    
    sld_base = "<RasterSymbolizer>  <ChannelSelection>   <GrayChannel>   <SourceChannelName>{channel}</SourceChannelName> </GrayChannel>        </ChannelSelection>    <ColorMap  type=\"ramp\" extended=\"false\" >  <ColorMapEntry color=\"#6baed6\" quantity=\"{quantity}\" />  </ColorMap></RasterSymbolizer>"

    return {
        "name":'Areas Likely Flooded with Sea Level Rise of {} {}'.format(amt, 'foot' if amt==1 else 'feet'),
        "application":["rw"],"iso":[],
        "default":True if ix==0 else False,
        "protected":False,
        "layerConfig":{
            "type":"gee",
            "assetId":'users/resourcewatch/cli_024_sea_level_rise_ft',
            "body":{
                "styleType":"sld",
                "sldValue":sld_base.format(channel=ix, quantity=amt)
            }},
        "legendConfig":{},
        "interactionConfig":{},
        "applicationConfig":{},
        "staticImageConfig":{},
    }

def upload_SLR_layers(uniq_id, amts):
    layer_defs = list(map(lambda item: slr_layer(item[0],item[1]), enumerate(amts)))
    #return layer_defs
    return list(map(lambda l_def: upload_layer_def_to_backoffice(l_def, uniq_id), layer_defs))

In [34]:
uniq_id = 'e4946b37-ac8f-48f7-9498-dee5ab23db68'
amts = [1,2,3,4,5,6,7,8,9,10]
upload_SLR_layers(uniq_id, amts)

['Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized',
 'Unauthorized']

In [61]:
rasters_uploaded = reduce(prepare_data, ic_upload_config.values, {})

INFO:root:{"data":{"id":"cf8218cf-4002-4d6f-80fb-9b34c85b008a","type":"dataset","attributes":{"name":"foo_051_Vegetation Condition Index (NDVI anomalies)","slug":"foo_051_Vegetation-Condition-Index-NDVI-anomalies","type":"raster","subtitle":null,"application":["rw"],"dataPath":null,"attributesPath":null,"connectorType":"rest","provider":"gee","userId":"58f66f96733af88e4f563967","connectorUrl":null,"tableName":"users/resourcewatch_wri/foo_051_vegetation_condition_index","status":"pending","published":true,"overwrite":false,"verified":false,"blockchain":{},"mainDateField":null,"env":"production","geoInfo":true,"protected":false,"legend":{"date":[],"region":[],"country":[],"nested":[]},"clonedHost":{},"errorMessage":null,"taskId":null,"updatedAt":"2018-03-30T13:47:02.448Z","widgetRelevantProps":[],"layerRelevantProps":[]}}}


In [57]:
def gen_lookup_links(info):
    rw_id = info[0]
    obj_info = info[1]
    wri_id = obj_info['wri_id']
    layer_id = obj_info['layers']['data']['id']
    url = 'https://staging-api.globalforestwatch.org/layer/{}/tile/gee/'.format(layer_id)
    return (wri_id, rw_id, url + '0/0/0')

In [62]:
list(map(gen_lookup_links, rasters_uploaded.items()))

[('foo_051',
  'cf8218cf-4002-4d6f-80fb-9b34c85b008a',
  'https://staging-api.globalforestwatch.org/layer/ef5729f1-5ee1-43c2-aca7-f55df2b4e0e3/tile/gee/0/0/0')]

In [39]:
def remove_dataset_and_layers(wri_id):
    create_res = req.request("GET", 
                      'https://staging-api.globalforestwatch.org/v1/dataset/{}/layer'.format(wri_id))
    layers = create_res.json()['data']
    layer_ids = [l['id'] for l in layers]
        
    deleted_layers = []
    for l_id in layer_ids:
        delete_layer = req.request("DELETE", 
                      'https://staging-api.globalforestwatch.org/v1/dataset/{}/layer/{}'.format(wri_id, l_id),
                       headers = createHeaders())
        
        deleted_layers.append(delete_layer.text)
    
    deleted_dataset = req.request("DELETE", 
                      'https://staging-api.globalforestwatch.org/v1/dataset/{}'.format(wri_id),
                       headers = createHeaders())
    
    
    return deleted_dataset.text, deleted_layers

deleted_info = list(map(remove_dataset_and_layers, raster_uploaded.keys()))

In [36]:
deleted_info

[('{"data":{"id":"c37cb3ec-919e-45a1-b2cb-ba8ae7bd6f7d","type":"dataset","attributes":{"name":"cli_005a_Antarctic Sea Ice","slug":"cli_005a_Antarctic-Sea-Ice","type":"raster","subtitle":null,"application":[],"dataPath":null,"attributesPath":null,"connectorType":"rest","provider":"gee","userId":"58f66f96733af88e4f563967","connectorUrl":null,"tableName":"users/resourcewatch_wri/cli_005_antarctic_sea_ice_extent_reproj","status":"saved","published":true,"overwrite":false,"verified":false,"blockchain":{},"mainDateField":null,"env":"production","geoInfo":true,"protected":false,"legend":{"date":[],"region":[],"country":[],"nested":[]},"clonedHost":{},"errorMessage":"","taskId":null,"updatedAt":"2018-03-30T13:05:43.834Z","widgetRelevantProps":[],"layerRelevantProps":[]}}}',
  ['{"data":{"id":"830cc0a9-ec07-4fee-90f3-6b505782f6a6","type":"layer","attributes":{"name":"Most recent Antarctic Sea Ice","slug":"Most-recent-Antarctic-Sea-Ice","dataset":"c37cb3ec-919e-45a1-b2cb-ba8ae7bd6f7d","applicati